In [161]:
import pandas as pd 
import numpy as np
import re
import nltk 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
train_data = pd.read_csv(r'/Users/toxicdataset/train.csv') 
test_data = pd.read_csv(r'/Users/toxicdataset/test.csv') 
# Preview the first 5 lines of the loaded data 
train_data.head()
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [162]:
train_colms = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [163]:
train_data.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [164]:
unlabelled_comments = train_data[(train_data['toxic']!=1) & (train_data['severe_toxic']!=1) & (train_data['obscene']!=1) & 
                            (train_data['threat']!=1) & (train_data['insult']!=1) & (train_data['identity_hate']!=1)]
print('Percentage of unlabelled comments is ', len(unlabelled_comments)/len(train_data)*100)

Percentage of unlabelled comments is  89.83211235124176


In [165]:
null_cmt = train_data[train_data['comment_text'].isnull()]
len(null_cmt)

0

In [166]:
train_data.isnull().any(),test_data.isnull().any()

(id               False
 comment_text     False
 toxic            False
 severe_toxic     False
 obscene          False
 threat           False
 insult           False
 identity_hate    False
 dtype: bool, id              False
 comment_text    False
 dtype: bool)

In [167]:
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [168]:
null_cmt = test_data[test_data['comment_text'].isnull()]
null_cmt

,id,comment_text


In [169]:
y = train_data[train_colms].values

In [170]:
def remove_stop_words(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [171]:
# clean the comment_text in train_data
train_data['comment_text'] = train_data['comment_text'].map(lambda c : remove_stop_words(c))

In [172]:
test_data['comment_text'] = test_data['comment_text'].map(lambda c : remove_stop_words(c))

In [173]:
train_data['total_length'] = train_data['comment_text'].apply(len)
train_data['capitals'] = train_data['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
train_data['caps_vs_length'] = train_data.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
train_data['num_exclamation_marks'] = train_data['comment_text'].apply(lambda comment: comment.count('!'))
train_data['num_question_marks'] = train_data['comment_text'].apply(lambda comment: comment.count('?'))
train_data['num_punctuation'] = train_data['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in '.,;:'))
train_data['num_symbols'] = train_data['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in '*&$%'))
train_data['num_words'] = train_data['comment_text'].apply(lambda comment: len(comment.split()))
train_data['num_unique_words'] = train_data['comment_text'].apply(
    lambda comment: len(set(w for w in comment.split())))
train_data['words_vs_unique'] = train_data['num_unique_words'] / train_data['num_words']
train_data['num_smilies'] = train_data['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

In [174]:
X = train_data.comment_text
test_X = test_data.comment_text

In [175]:
print(X.shape, test_X.shape)

(159571,) (153164,)


In [176]:
159571/(159571+153164) # 51% data for training and 49% on model deployment/testing

0.5102434968903384

In [177]:
# import and instantiate TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [178]:
# learn the vocabulary in the training data, then use it to create a document-term matrix
X_dtm = vect.fit_transform(X)
# examine the document-term matrix created from X_train
X_dtm

<159571x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 3178792 stored elements in Compressed Sparse Row format>

In [179]:
# transform the test data using the earlier fitted vocabulary, into a document-term matrix
test_X_dtm = vect.transform(test_X)
# examine the document-term matrix from X_test
test_X_dtm

<153164x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2618972 stored elements in Compressed Sparse Row format>

In [181]:
from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=3)

# train
classifier.fit(train_data, train_data)

# predict
predictions = classifier.predict(test_data)

AttributeError: 'DataFrame' object has no attribute 'getformat'